<a href="https://colab.research.google.com/github/tsitsvero/rotor-gp/blob/main/pack/train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [36]:
!pip install git+https://github.com/lab-cosmo/librascal gpytorch pytorch-lightning wandb xtb nbstripout
!pip install git+https://github.com/chem-gp/fande --force-reinstall
# ! nbstripout --install --global

  Cloning https://github.com/lab-cosmo/librascal to /tmp/pip-req-build-ubbxnr28
  Running command git clone --filter=blob:none --quiet https://github.com/lab-cosmo/librascal /tmp/pip-req-build-ubbxnr28
  Resolved https://github.com/lab-cosmo/librascal to commit 6c55e99720f9a181f1efc5c7cd4976c87809d79b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/chem-gp/fande to /tmp/pip-req-build-vsccjk_a
  Running command git clone --filter=blob:none --quiet https://github.com/chem-gp/fande /tmp/pip-req-build-vsccjk_a
  Resolved https://github.com/chem-gp/fande to commit aded96b5f6a1eec9b699457a47248bcb58f290d1
  Preparing metadata (setup.py) ... done


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Cooking up the model

In [38]:
%%time
import os
# DATA_DIR = os.path.expanduser("~/repos/data/")
# # DATA_DIR = "/data1/simulations/datasets/rotors/high_temp_ML_training_data/"
# RESULTS_DIR = os.path.expanduser("~/repos/data/results")


DATA_DIR = os.path.expanduser("/content/drive/MyDrive/data/")
# FANDE_DIR = os.path.expanduser("~/")
RESULTS_DIR = os.path.expanduser("/content/results")
# os.makedirs(RESULTS_DIR, exist_ok=True)


ENERGY_MODEL = 'exact' #'variational_inducing_points', 'exact'
ENERGY_NUM_INDUCING_POINTS = 500
ENERGY_LR = 0.01
ENERGY_NUM_STEPS = 2000

FORCES_MODEL = 'exact' #'variational_inducing_points', 'exact'
FORCES_NUM_INDUCING_POINTS = 500
NUM_FORCE_SAMPLES = 1000
FORCES_LR = 0.01
FORCES_NUM_STEPS = 2000

PREDICTOR_NAME = 'test.pth'
SUBSAMPLE = 2 # subsample data to reduce time durings tests

! python cook_model.py \
--data_dir $DATA_DIR \
--results_dir $RESULTS_DIR \
--energy_model $ENERGY_MODEL \
--energy_num_inducing_points $ENERGY_NUM_INDUCING_POINTS \
--energy_lr $ENERGY_LR \
--energy_num_steps $ENERGY_NUM_STEPS \
--forces_model $FORCES_MODEL \
--forces_num_inducing_points $FORCES_NUM_INDUCING_POINTS \
--num_force_samples $NUM_FORCE_SAMPLES \
--forces_lr $FORCES_LR \
--forces_num_steps $FORCES_NUM_STEPS \
--predictor_name $PREDICTOR_NAME \
--subsample $SUBSAMPLE


DATA_DIR /content/drive/MyDrive/data/
RESULTS_DIR /content/results
ENERGY_MODEL exact
ENERGY_NUM_INDUCING_POINTS 500
ENERGY_LR 0.01
ENERGY_NUM_STEPS 2000
FORCES_MODEL exact
FORCES_NUM_INDUCING_POINTS 500
NUM_FORCE_SAMPLES 1000
FORCES_LR 0.01
FORCES_NUM_STEPS 2000
PREDICTOR_NAME test.pth
500 500
fande.models module imported...
Icecream logger is not available
Total number of found groups: 14
Checking if all atoms are covered:  True
Total length of traj is 500
Total number of batches 500
Calculating invariants on trajectory with librascal...
100% 500/500 [00:13<00:00, 36.66it/s]
invariants for energy fitting calculated
100% 500/500 [09:03<00:00,  1.09s/it]
invariants for forces fitting calculated
torch.Size([500, 1200])
torch.Size([500])
Dataloader for group 0 created
Number of samples in dataloader: 1000
Dataloader for group 1 created
Number of samples in dataloader: 1000
Dataloader for group 2 created
Number of samples in dataloader: 1000
Dataloader for group 3 created
Number of sample

## Testing models

In [39]:
from ase import io
test_traj = io.read(DATA_DIR + "/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="1000:1010")

In [40]:
import sys
import torch
FANDE_DIR = os.path.expanduser("~/repos/")
sys.path.append(FANDE_DIR + "fande/")

from fande.predict import FandePredictor
from fande.ase import FandeCalc
# load the predictor:
# predictor_loaded = torch.load(RESULTS_DIR + "/fande_predictor.pth")
predictor_loaded = torch.load(RESULTS_DIR + "/test.pth")
fande_calc_loaded = FandeCalc(predictor_loaded)
device = torch.device('cpu')
fande_calc_loaded.predictor.move_models_to_device(device)

In [41]:
forces_true = []
forces_predicted = []

energy_true = []
energy_predicted = []

for atoms in test_traj:
    forces_true.append(atoms.get_forces())
    energy_true.append(atoms.get_potential_energy())
    atoms.set_calculator(fande_calc_loaded)
    forces_predicted.append(atoms.get_forces())
    energy_predicted.append(atoms.get_potential_energy())

INFO:fande:Setting context for descriptors calculation to production
INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1366.1811351776123
Predicting forces...
Energy model summary: 
Time invariants:  0.23984909057617188
Time prediction:  100.20017623901367
Time moving on device:  0.732421875
Time total:  109.12299156188965


INFO:fande:Setting context for descriptors calculation to production
INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1699.1944313049316
Predicting forces...
Energy model summary: 
Time invariants:  0.1990795135498047
Time prediction:  9.5062255859375
Time moving on device:  1.4545917510986328
Time total:  14.257192611694336
Time for invariants (call from forces):  1776.6029834747314
Predicting forces...
Energy model summary: 
Time invariants:  0.18525123596191406
Time prediction:  24.628639221191406
Time moving on device:  0.8378028869628906
Time total:  40.886878967285156


INFO:fande:Setting context for descriptors calculation to production


...

Energy model summary: 
Time invariants:  0.15783309936523438
Time prediction:  112.13397979736328
Time moving on device:  0.8337497711181641
Time total:  124.3743896484375
Time for invariants (call from forces):  1676.2523651123047
Predicting forces...


INFO:fande:Setting context for descriptors calculation to production
INFO:fande:Setting context for descriptors calculation to production


Energy model summary: 
Time invariants:  0.18930435180664062
Time prediction:  74.32031631469727
Time moving on device:  6.214380264282227
Time total:  105.34095764160156
Time for invariants (call from forces):  1175.8182048797607
Predicting forces...
Energy model summary: 
Time invariants:  0.2951622009277344
Time prediction:  9.64212417602539
Time moving on device:  0.8225440979003906
Time total:  18.388032913208008


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1725.3327369689941
Predicting forces...
Energy model summary: 
Time invariants:  0.1900196075439453
Time prediction:  94.63310241699219
Time moving on device:  1.196146011352539
Time total:  106.75287246704102
Time for invariants (call from forces):  1583.681583404541
Predicting forces...
Energy model summary: 
Time invariants:  0.19121170043945312
Time prediction:  83.96315574645996
Time moving on device:  0.7512569427490234
Time total:  94.06495094299316


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

energy_true = np.array(energy_true)
energy_predicted = np.array(energy_predicted)
energy_errors = energy_predicted - energy_true

forces_true = np.array(forces_true)
forces_predicted = np.array(forces_predicted)
forces_errors = forces_predicted - forces_true

atomic_groups = predictor_loaded.fdm.atomic_groups



for ag in atomic_groups:

    print("Atomic group", ag)
    print("F_x")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 0].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 0].flatten()**2)))
    print("F_y")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 1].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 1].flatten()**2)))
    print("F_z")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 2].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 2].flatten()**2)))
    print("E")
    print("MAE", np.mean(np.abs(energy_errors.flatten())))
    print("RMSE", np.sqrt(np.mean(energy_errors.flatten()**2)))

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))
    axs[0].set_title(r"$F_x$")
    axs[0].plot(np.array(forces_true)[:, ag, 0].flatten(), label="true")
    axs[0].plot(np.array(forces_predicted)[:, ag, 0].flatten(), label="predicted")
    axs[0].legend()

    axs[1].set_title(r"$F_y$")
    axs[1].plot(np.array(forces_true)[:, ag, 1].flatten(), label="true")
    axs[1].plot(np.array(forces_predicted)[:, ag, 1].flatten(), label="predicted")
    axs[1].legend()

    axs[2].set_title(r"$F_z$")
    axs[2].plot(np.array(forces_true)[:, ag, 2].flatten(), label="true")
    axs[2].plot(np.array(forces_predicted)[:, ag, 2].flatten(), label="predicted")
    axs[2].legend()

    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))

    axs[0].set_title(r"$F_x$ errors")
    axs[0].hist(forces_errors[:, ag, 0].flatten(), bins=100)

    axs[1].set_title(r"$F_y$ errors")
    axs[1].hist(forces_errors[:, ag, 1].flatten(), bins=100)

    axs[2].set_title(r"$F_z$ errors")
    axs[2].hist(forces_errors[:, ag, 2].flatten(), bins=100)
    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
    axs[0].set_title(r"$E$ errors")
    axs[0].hist(energy_errors.flatten(), bins=100)

    axs[1].set_title(r"$E$")
    axs[1].plot(energy_true, label="true")
    axs[1].plot(energy_predicted, label="predicted")
    axs[1].legend()
    plt.show()
